# Set Working Directory

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [3]:
from langchain_neo4j import Neo4jGraph
# from app.graph_builders.micro_graph_builder import MicroCodeGraphBuilder
url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(
    url=url,
    username=username,
    password=password,
    enhanced_schema=True
)

In [7]:
print(graph_db.get_schema)

Node properties:
- **Class**
  - `id`: STRING Available options: ['Langchain_Core.Messages.Aimessage', 'Langchain_Core.Messages.Humanmessage']
- **Module**
  - `id`: STRING Example: "App.Handlers.Process_Query"
- **Package**
  - `id`: STRING Example: "Atexit"
- **Function**
  - `id`: STRING Available options: ['Src.Streamlit_App.Logger', 'Src.Streamlit_App.Set_Log_Level', 'Src.Streamlit_App.Main', 'Src.App.Handlers.Query_Handler.Process_Query', 'Src.App.Ui.Display_Messages', 'Src.App.Setup.Initialize_Database.Initialize_Datab', 'Src.App.Setup.Initialize_Database']
Relationship properties:

The relationships:
(:Class)-[:USES]->(:Function)
(:Module)-[:USED_IN]->(:Module)
(:Module)-[:DEFINES]->(:Function)
(:Module)-[:USES]->(:Function)
(:Module)-[:IMPORTS]->(:Function)
(:Module)-[:IMPORTS]->(:Package)
(:Module)-[:DEFINES_FUNCTION]->(:Function)
(:Package)-[:USED_IN]->(:Module)
(:Package)-[:USES]->(:Function)
(:Package)-[:USES]->(:Module)
(:Function)-[:DEFINED_IN]->(:Module)
(:Function)-[:U

In [15]:
graph_db.refresh_schema()

In [ ]:
results = graph_db.query("""MATCH path = (m1:Module {id: 'App.Handlers'})-[*..10]-(m2:Module {id: 'Streamlit_App'}) 
               WHERE NONE(n IN nodes(path) WHERE n:Package) 
               RETURN [n IN nodes(path) | labels(n)] AS nodeTypes, 
                      [r IN relationships(path) | type(r)] AS relTypes, 
                      path
               LIMIT 50""")
results

[]

In [31]:
results = graph_db.query("""
MATCH path = (a)-[*..10]-(b)
WHERE NONE(n IN nodes(path) WHERE n:Package)
WITH nodes(path) AS ns, relationships(path) AS rs
UNWIND range(0, size(ns) - 2) AS i
RETURN 
  head(labels(ns[i])) AS fromNodeType,
  ns[i].id AS fromNodeId,
  type(rs[i]) AS relationType,
  head(labels(ns[i+1])) AS toNodeType,
  ns[i+1].id AS toNodeId
LIMIT 50""")

In [32]:
results

[{'fromNodeType': 'Module',
  'fromNodeId': 'App.Handlers.Process_Query',
  'relationType': 'USED_IN',
  'toNodeType': 'Module',
  'toNodeId': 'Src.Streamlit_App'},
 {'fromNodeType': 'Module',
  'fromNodeId': 'App.Handlers.Process_Query',
  'relationType': 'USED_IN',
  'toNodeType': 'Module',
  'toNodeId': 'Src.Streamlit_App'},
 {'fromNodeType': 'Module',
  'fromNodeId': 'Src.Streamlit_App',
  'relationType': 'USED_IN',
  'toNodeType': 'Function',
  'toNodeId': 'Src.Streamlit_App.Main'},
 {'fromNodeType': 'Module',
  'fromNodeId': 'App.Handlers.Process_Query',
  'relationType': 'USED_IN',
  'toNodeType': 'Module',
  'toNodeId': 'Src.Streamlit_App'},
 {'fromNodeType': 'Module',
  'fromNodeId': 'Src.Streamlit_App',
  'relationType': 'USED_IN',
  'toNodeType': 'Function',
  'toNodeId': 'Src.Streamlit_App.Main'},
 {'fromNodeType': 'Function',
  'fromNodeId': 'Src.Streamlit_App.Main',
  'relationType': 'DEFINED_IN',
  'toNodeType': 'Module',
  'toNodeId': 'Src.Streamlit_App'},
 {'fromNodeTy

# Preprocessing data (Default when repository is given)

## Create GRAPHRAG DB / Preprocess and insert Data

In [33]:
from app.graph_builders.micro_graph_builder import MicroCodeGraphBuilder
from langchain_anthropic import ChatAnthropic
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o", model_provider="openai")
# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

gen_graphdb = MicroCodeGraphBuilder(llm=llm)
graph_db, graph_documents = gen_graphdb.build_graph_and_upload(repo_path='../cloned_repository')

2025-04-26 06:44:08.411 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Processing file: src\streamlit_app.py
Processing file: src\agent_router\graph.py
Processing file: src\agent_router\nodes.py
Processing file: src\agent_router\state.py
Processing file: src\agent_router\__init__.py
Processing file: src\app\session_state.py
Processing file: src\app\__init__.py
Processing file: src\app\graph_builders\micro_graph_builder.py
Processing file: src\app\graph_builders\__init__.py
Processing file: src\app\handlers\query_handler.py
Processing file: src\app\handlers\__init__.py
Processing file: src\app\setup\initialize_database.py
Processing file: src\app\setup\__init__.py
Processing file: src\app\ui\chat.py
Processing file: src\app\ui\visualization.py
Processing file: src\app\ui\__init__.py
Processing file: src\rag\vector_rag.py
Processing file: src\rag\__init__.py
Processing file: src\tools\setup.py
Processing file: src\tools\__init__.py
Processing file: src\tools\macro\tools.py
Processing file: src\tools\macro\__init__.py
Processing file: src\tools\micro\cypher_

In [34]:
graph_documents

[GraphDocument(nodes=[Node(id='Atexit', type='Package', properties={}), Node(id='Os', type='Package', properties={}), Node(id='Shutil', type='Package', properties={}), Node(id='Subprocess', type='Package', properties={}), Node(id='Time', type='Package', properties={}), Node(id='Uuid', type='Package', properties={}), Node(id='Functools', type='Package', properties={}), Node(id='Typing', type='Package', properties={}), Node(id='Streamlit', type='Package', properties={}), Node(id='Langchain.Chat_Models', type='Package', properties={}), Node(id='Langchain_Anthropic', type='Package', properties={}), Node(id='Langchain_Core.Messages', type='Package', properties={}), Node(id='Src.App.Handlers.Query_Handler.Process_Query', type='Function', properties={}), Node(id='Src.App.Session_State.Cleanup_Repo', type='Function', properties={}), Node(id='Src.App.Session_State.Initialize_Session_State', type='Function', properties={}), Node(id='Src.App.Setup.Initialize_Database.Initialize_Database', type='F

In [43]:
results = graph_db.query("""MATCH path = (m1:Module {id: 'Src.App.Handlers.Query_Handler'})-[*..5]-(m2:Module {id: 'Src.Streamlit_App'}) WHERE NONE(n IN nodes(path) WHERE n:Package) RETURN [n IN nodes(path) | labels(n)] AS nodeTypes, path LIMIT 20""")
results

[{'nodeTypes': [['Module'],
   ['Function'],
   ['Module'],
   ['Function'],
   ['Function'],
   ['Module']],
  'path': [{'id': 'Src.App.Handlers.Query_Handler'},
   'DEFINED_IN',
   {'id': 'Src.App.Handlers.Query_Handler.Process_Query'},
   'USED_IN',
   {'id': 'Src.Tools.Micro.Tools'},
   'USES',
   {'id': 'Src.App.Setup.Initialize_Database.Initialize_Database'},
   'USES',
   {'id': 'Src.Streamlit_App.Main'},
   'CALLS',
   {'id': 'Src.Streamlit_App'}]},
 {'nodeTypes': [['Module'],
   ['Function'],
   ['Module'],
   ['Function'],
   ['Function'],
   ['Module']],
  'path': [{'id': 'Src.App.Handlers.Query_Handler'},
   'DEFINED_IN',
   {'id': 'Src.App.Handlers.Query_Handler.Process_Query'},
   'USED_IN',
   {'id': 'Src.Tools.Micro.Tools'},
   'USES',
   {'id': 'Src.App.Setup.Initialize_Database.Initialize_Database'},
   'USES',
   {'id': 'Src.Streamlit_App.Main'},
   'DEFINED_IN',
   {'id': 'Src.Streamlit_App'}]},
 {'nodeTypes': [['Module'],
   ['Function'],
   ['Function'],
   ['Func

In [40]:
from typing import List, Dict, Any

def visualize_cypher_results(cypher_results: List[Dict[str, Any]], output_file: str = "cypher_graph.html"):
    from pyvis.network import Network
    import networkx as nx
    import matplotlib.pyplot as plt

    net = Network(notebook=False, cdn_resources='in_line', height="800px", width="100%")
    G = nx.DiGraph()

    unique_nodes = set()
    unique_edges = set()
    node_metadata = {}

    for result in cypher_results:
        path = result.get('path', [])
        node_types = result.get('nodeTypes', [])

        node_idx = 0  # Index to track nodeTypes separately
        
        for i in range(0, len(path), 2):
            if i < len(path):
                node = path[i]
                node_id = node.get('id')

                if node_id and node_id not in unique_nodes:
                    unique_nodes.add(node_id)

                    # Grab the nodeType from nodeTypes list
                    if node_idx < len(node_types):
                        node_meta_type = node_types[node_idx][0]  # nodeTypes is a list of list
                    else:
                        node_meta_type = "Unknown"

                    node_metadata[node_id] = {
                        "id": node_id,
                        "meta": node_meta_type,
                        "properties": node.get('properties', {})
                    }

                    G.add_node(node_id)

                node_idx += 1  # Move to the next nodeType

            if i + 1 < len(path) and i + 2 < len(path):
                source_id = path[i].get('id')
                relationship = path[i + 1]
                target_id = path[i + 2].get('id')

                if source_id and target_id:
                    edge_key = (source_id, target_id, relationship)
                    if edge_key not in unique_edges:
                        unique_edges.add(edge_key)
                        G.add_edge(source_id, target_id, label=relationship)

    # Group nodes by their meta type
    unique_metas = list(set(meta["meta"] for meta in node_metadata.values()))
    color_map = plt.get_cmap("tab10")
    meta_colors = {m: color_map(i / len(unique_metas)) for i, m in enumerate(unique_metas)}
    meta_colors_rgba = {
        m: f'rgba({int(c[0]*255)}, {int(c[1]*255)}, {int(c[2]*255)}, 0.8)'
        for m, c in meta_colors.items()
    }

    degrees = dict(G.degree())
    min_size, max_size = 15, 50
    max_degree = max(degrees.values()) if degrees else 1
    size_scale = {
        node: min_size + (max_size - min_size) * (deg / max_degree)
        for node, deg in degrees.items()
    }

    for node_id in G.nodes():
        metadata = node_metadata.get(node_id, {})
        node_meta = metadata.get("meta", "Unknown")
        color = meta_colors_rgba.get(node_meta, "gray")

        net.add_node(
            node_id,
            label=node_id,
            size=size_scale[node_id],
            color=color,
            title=f"<b>{node_meta}</b><br>{node_id}"
        )

    relationship_types = list(set(attr.get("label", "") for _, _, attr in G.edges(data=True)))
    rel_color_map = plt.get_cmap("Set2")
    rel_colors = {
        rel_type: f'rgba({int(rel_color_map(i / len(relationship_types))[0]*255)}, {int(rel_color_map(i / len(relationship_types))[1]*255)}, {int(rel_color_map(i / len(relationship_types))[2]*255)}, 0.9)'
        for i, rel_type in enumerate(relationship_types)
    }

    for source, target, attr in G.edges(data=True):
        rel_label = attr.get("label", "")
        edge_color = rel_colors.get(rel_label, "#888888")
        net.add_edge(source, target, title=rel_label, label=rel_label, arrows='to', length=300, color=edge_color)

    net.save_graph(output_file)

    # Build the correct legend
    legend_html = """
    <div id="legend" style="position: absolute; top: 10px; left: 10px; background: white; padding: 10px; border-radius: 8px; box-shadow: 0px 0px 5px rgba(0,0,0,0.2); font-family: Arial, sans-serif; z-index: 1000;">
        <h4 style="margin: 0; padding-bottom: 5px;">Node Legend (by Node Type)</h4>
    """
    for meta_type, color in meta_colors_rgba.items():
        legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 5px;"><div style="width: 15px; height: 15px; background:{color}; margin-right: 5px; border-radius: 50%;"></div> {meta_type}</div>'

    legend_html += """
        <h4 style="margin: 5px 0; padding-bottom: 5px;">Relationship Legend</h4>
    """
    for rel_type in sorted(relationship_types):
        rel_color = rel_colors.get(rel_type, "#888888")
        legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 5px;"><div style="width: 20px; height: 3px; background: {rel_color}; margin-right: 5px;"></div> {rel_type}</div>'

    legend_html += "</div>"

    with open(output_file, "r", encoding="utf-8") as file:
        html_content = file.read()

    html_content = html_content.replace("</body>", legend_html + "</body>")

    with open(output_file, "w", encoding="utf-8") as file:
        file.write(html_content)

    print(f"Graph visualization saved as {output_file}")

    return {
        "nodes": len(G.nodes()),
        "edges": len(G.edges()),
        "node_types": unique_metas,
        "relationship_types": list(set(attr.get("label", "") for _, _, attr in G.edges(data=True)))
    }


In [42]:
visualize_cypher_results(results)

Graph visualization saved as cypher_graph.html


{'nodes': 7,
 'edges': 9,
 'node_types': ['Module', 'Function'],
 'relationship_types': ['USES', 'DEFINED_IN', 'USED_IN']}

In [11]:
print(graph_documents[0].source.page_content)

1. math is a package and is used in all the methods of the class deduplication.bloom_filter.BloomFilter.
2. mmh3 is a package and is used in the method deduplication.bloom_filter.BloomFilter.add of the class deduplication.bloom_filter.BloomFilter.
3. bitarray is a package and is used in the method deduplication.bloom_filter.BloomFilter.__init__ of the class deduplication.bloom_filter.BloomFilter.
4. nltk is a package and is used in the method deduplication.bloom_filter.BloomFilter.add of the class deduplication.bloom_filter.BloomFilter.
5. random is a package and is used in the method deduplication.bloom_filter.BloomFilter_Uni_Hash.__init__ of the class deduplication.bloom_filter.BloomFilter_Uni_Hash.
6. deduplication.bloom_filter.BloomFilter.add is a method with properties defined in deduplication.bloom_filter.BloomFilter which is a class.
7. deduplication.bloom_filter.BloomFilter.query is a method with properties defined in deduplication.bloom_filter.BloomFilter which is a class.
8. 

### Reset Instance

In [9]:
# graph_db.query("MATCH (n) DETACH DELETE n")

## Visualize Entire Graph

In [4]:
a = gen_graphdb.create_visualization(graph_documents = graph_documents)

Graph saved as semantic_graph.html


## Create Vector DB / Preprocess and insert Data

In [ ]:
from rag.vector_rag import create_rag_system

db_setup, query_engine = create_rag_system(
    # repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    repo_path="../cloned_repository",
    embedding_model="text-embedding-3-large",
    llm_model="gpt-4o",
)


# Demonstrate Tools

## Cypher Query to generate initial Answer

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [2]:
# from langchain_core.language_models import BaseChatModel
# from langchain_neo4j import Neo4jGraph
# from tools.micro_cypher_chain import CypherGraphBuilder

# url = os.getenv('NEO4J_URI')
# username = os.getenv('NEO4J_USERNAME')
# password = os.getenv('NEO4J_PASSWORD')

# graph_db = Neo4jGraph(
#     url=url,
#     username=username,
#     password=password,
#     enhanced_schema=True
#     )

# def retrieve_cypher_relationships(question: str):
#     """
#     Connects to a local Neo4j instance and uses a language model to generate a Cypher-based graph
#     representation of relationships between entities (e.g., code modules).

#     Returns:
#         dict: A structured representation of nodes and relationships inferred from the graph.
#     """

#     # Create the Cypher graph
#     builder = CypherGraphBuilder(llm=llm, graph_db=graph_db)
#     # cypher_graph = builder.create_cypher_graph()
#     cypher_graph = builder.create_cypher_graph()
#     result = cypher_graph.invoke({"question": question})

#     return result['answer']

In [20]:
from langchain_neo4j import Neo4jGraph
from tools.micro_cypher_chain import CypherGraphBuilder
from langchain_anthropic import ChatAnthropic

url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(
    url=url,
    username=username,
    password=password,
    enhanced_schema=True
)


# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")


builder = CypherGraphBuilder(llm=llm, graph_db=graph_db)
cypher_graph = builder.create_cypher_graph()

In [21]:
result = cypher_graph.invoke({"question": "How is the Utils.Utils module related to the Deduplication.__Main__ module?"})

In [23]:
result['cypher_statement']

"MATCH path = (m:Module {id: 'Utils.Utils'})-[*..10]-(c:Module {id: 'Deduplication.__Main__'}) RETURN path LIMIT 10"

In [5]:
result['answer']

'The `Utils.Utils` module is related to the `Deduplication.__Main__` module in the following ways:\n\n- **Uses Minhash**: `Utils.Utils` imports `Utils.Utils.Minhash`, which is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and is imported into `Deduplication.__Main__`.\n- **Uses Shingle**: `Utils.Utils` imports `Utils.Utils.Shingle`, which is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and is imported into `Deduplication.__Main__`.\n- **Uses Clean Document**: `Utils.Utils` imports `Utils.Utils.Clean_Document`, which is used in `Utils.Use_Cases.Nearest_Neighbor_Search` and is imported into `Deduplication.__Main__`.\n- **Reads TSV**: `Utils.Utils` defines `Utils.Utils.Read_Tsv`, which is used in `Deduplication.__Main__`.\n- **Used in**: Definitions from `Utils.Utils` that involve `Utils.Utils.Read_Tsv` are specifically indicated as being used in `Deduplication.__Main__` and `Deduplication.__Main__.Main`.'

In [6]:
result['cypher_results']

[{'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind'},
   'DEFINES',
   {'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Minhash'},
   'USES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'IMPORTED_IN',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind'},
   'DEFINES',
   {'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Minhash'},
   'USES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'USED_IN',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind'},
   'DEFINES',
   {'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Shingle'},
   'USES',
   {'id': 'Utils.Use_Cases.Nearest_Neighbor_Search'},
   'IMPORTED_IN',
   {'id': 'Deduplication.__Main__'}]},
 {'path': [{'id': 'Utils.Utils'},
   'IMPORTED_FROM',
   {'id': 'Utils.Utils.Unionfind

In [7]:
# graph_db.query("""
# MATCH path = (m1:Module {id: 'Utils.Utils'})-[*..5]-(m2:Module {id: 'Deduplication.__Main__'}) 
# WHERE NONE(n IN nodes(path) WHERE n:Package)
# RETURN path
# """)

In [13]:
# import os
# print(os.getcwd())
# os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
# print(os.getcwd())

# from tools.micro_tools import (retrieve_cypher_relationships)

# cypher_generator = retrieve_cypher_relationships("How is the Utils.Utils module related to the Deduplicatioin.__Main__ module?")

## Final Text Response(VectorDB)

In [9]:
questions = """
The Utils.Utils module is intricately related to the Deduplication.__Main__ module through several intermediate connections, reflecting a well-structured framework that enhances document deduplication capabilities. Here are the key points of this relationship:

1. **Function Utilization**: The Utils.Utils module utilizes the `Compute_Minhash_Signatures` function from Deduplication.Lsh.Lsh, which plays a crucial role in generating MinHash signatures for documents. This computational function is essential for the efficiency of the deduplication process, as it allows for quick similarity comparisons between documents based on their hashed representations.

2. **Class Interdependency**: The Deduplication.Lsh.Lsh module invokes the `Model` class from Deduplication.__Main__. This class serves as the backbone of the deduplication system, encapsulating core functionalities and configurations necessary for implementing various LSH-based methods effectively. The relationship emphasizes a modular design where the Lsh module extends the capabilities defined in the __Main__ module, facilitating code maintainability and scalability.

3. **Definition of Key Classes**: The Deduplication.__Main__ module independently defines both the `Model` class and the `__Main__` class. The `Model` class encompasses methods for executing the deduplication algorithms and managing the data structures involved, while the `__Main__` class is responsible for orchestrating the overall execution flow of the deduplication process. This design fosters a clear separation of concerns, allowing more straightforward debugging and enhancement in future iterations.

4. **Indirect Connections through Variants**: Additional relationships exist through other modules like `Lshimproved.Lsh` and `Lshforest.Lshforest`, which are specifically imported by Deduplication.__Main__. These imports signify that the main deduplication script leverages advancements in LSH techniques, such as improved processes for handling larger datasets and enhanced accuracy in identifying duplicate documents.

In summary, the Utils.Utils module is primarily connected to Deduplication.__Main__ through shared components and dependencies within the overall Deduplication system. These connections enhance the utility and efficiency of the deduplication process, particularly through the Lsh module and its associated classes and functions. The architecture aligns with best practices in software development by promoting modularity, enabling future enhancements like the implementation of new LSH-based algorithms without extensive modifications to the existing structure. This design not only streamlines the deduplication process but also positions the system for potential application in various scenarios, including plagiarism detection and content clustering.
"""
# Query the system
answer = query_engine.query(result['answer'])
print(answer)

Based on the provided sentence and context, I can add relevant details to make it more informative and useful:

The `Utils.Utils` module is a crucial component in the `Deduplication.__Main__` module, providing essential functionality for document processing and deduplication. Specifically:

1. The `Utils.Utils` module offers several key functions that are utilized in `Deduplication.__Main__`:

   - `Read_Tsv`: This function reads tab-separated values (TSV) files, likely used to load document datasets for deduplication.
   - `Minhash`: Implements the MinHash algorithm to create compact signatures of document shingles, essential for efficient similarity estimation between documents.
   - `Shingle`: Generates k-shingles from text, preserving some word order information for document comparison.
   - `Clean_Document`: Normalizes text by converting to lowercase and removing non-alphabetic characters, crucial for preprocessing before analysis.
   - `Majority_Vote`: Performs a majority vote on

## Final Visualization Response

In [10]:
from tools.micro_visual_response import visualize_cypher_results
stats = visualize_cypher_results(result['cypher_results'], "code_relationships_graph.html")

Graph visualization saved as code_relationships_graph.html


# Agent

In [1]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


from tools.micro_text_answer import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="gpt-4o",
)


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [2]:
from tools.micro_tools import (retrieve_cypher_relationships, visualize_relationships, generate_text_response)
from tools import micro_tools
from langgraph.prebuilt import create_react_agent

micro_tools.query_engine = query_engine

tools = [
    retrieve_cypher_relationships,
    visualize_relationships,
    generate_text_response,
]

system_prompt = f"""
You are a suupervisor that answers the questions given the following tools.
{tools}

You will use each tool only once and the guidelines of the tools are as follows:
- You will always use the retrieve_cypher_relationships tool to get an initial answer.
- You will always use the generate_text_response tool to get a final answer.
- Only use the visualize_relationships tool to visualize the relationships when the user asks for a visualization.

USE EACH TOOL ONLY ONCE.

Always provide clear, actionable responses with specific details and implementation steps.
"""

from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o", model_provider="openai")

agent = create_react_agent(llm, tools=tools, prompt=system_prompt)

In [3]:
answer = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "How is the Utils.Utils module related to the Deduplication.__Main__ module?",
            }
        ],
        "goto": "supervisor",
    }
)

importing tools


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


initializing Neo4jGraph
Building Cypher graph
done
done
Invoking cypher graph
Generating text response...
Results: {'answer': '- `Utils.Utils` defines `Utils.Utils.Majority_Vote`, which is imported into `Deduplication.Lshforest`, eventually making its way into `Deduplication.__Main__`.\n- `Utils.Utils` defines `Utils.Utils.Majority_Vote`, which is used in `Deduplication.Lshforest.Lshforest.Banding`, and this is imported into `Deduplication.__Main__`.\n- `Utils.Utils` defines `Utils.Utils.Majority_Vote`, which is part of `Deduplication.Lshforest.Lshforest`, extending `Deduplication.Lsh.Lsh` that is imported into `Deduplication.__Main__`.\n- `Utils.Utils` directly defines content (`Utils.Utils.Majority_Vote`) that is used by `Deduplication.__Main__`, specifically in its model usage.\n- `Utils.Utils.Read_Tsv` is defined in `Utils.Utils` and imported into `Deduplication.__Main__`.', 'steps': ['generate_cypher', 'execute_cypher', 'generate_final_answer'], 'cypher_statement': "MATCH path = (

In [4]:
print(answer['messages'][-1].content)

The `Utils.Utils` module and the `Deduplication.__Main__` module are connected through several relationships and dependencies within the codebase. Here is a detailed breakdown of their relationship:

1. **Definition and Usage**:
   - `Utils.Utils` defines a component called `Utils.Utils.Majority_Vote`. This component is crucial and is imported into various parts of the deduplication framework, including `Deduplication.Lshforest` and through to `Deduplication.__Main__`.
   - `Utils.Utils.Majority_Vote` is specifically used within `Deduplication.Lshforest.Lshforest.Banding`, which is part of the operations in the `Deduplication.__Main__`.

2. **Hierarchical Integration**:
   - The component `Utils.Utils.Majority_Vote` is integrated into `Deduplication.Lshforest`. The module `Deduplication.Lshforest` contains LSH (Locality-Sensitive Hashing) functionalities that extend its capabilities via the integration of majority voting systems for enhanced duplicate detection or elimination.
   - The

In [4]:
answer

{'messages': [HumanMessage(content='How is the Utils.Utils module related to the Deduplication.__Main__ module? Can you give me a visualization as well?', additional_kwargs={}, response_metadata={}, id='93292251-b958-4aaa-acda-f0956832283b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xumAZ62KiNB87z21VqqKcaU5', 'function': {'arguments': '{"question":"How is the Utils.Utils module related to the Deduplication.__Main__ module?"}', 'name': 'retrieve_cypher_relationships'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 731, 'total_tokens': 766, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_85bc8e3fae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-56e374d2-9391-4c

# Entire Agent Demo

In [6]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [5]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())


from rag.vector_rag import create_rag_system

db_setup, query_engine = create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="gpt-4o",
)

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [ ]:
# from agent_router.graph import create_router_graph
import agent_router.graph as agent_router
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chat_models import init_chat_model

import  tools.micro.tools as mt

import importlib
importlib.reload(mt)
importlib.reload(agent_router)

# import (retrieve_cypher_relationships, visualize_relationships, generate_text_response)

llm = init_chat_model("gpt-4o", model_provider="openai")

macro_tools = [
    # generate_repo_tree,
    # read_code_file,
    # list_python_files,
    # explain_code_logic,
]

# Add micro tools if available
mt.query_engine = query_engine

micro_tools = [
    mt.retrieve_cypher_relationships,
    mt.visualize_relationships,
    mt.generate_text_response,
]
graph = agent_router.create_router_graph(
            llm=llm, macro_tools=macro_tools, micro_tools=micro_tools
        )

2025-04-24 21:24:25 - agent_router - INFO - Building the router graph...
2025-04-24 21:24:25 - agent_router - INFO - Compiling the router graph...


In [9]:
import uuid
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

In [10]:
full_query = 'How is the Utils.Utils module related to the Deduplication.__Main__ module?'
# full_query = 'What does the Deduplication.__Main__ module do?'

# Add the user message to the conversation history
user_message = HumanMessage(content=full_query)

# Initialize the state with the user query
initial_state = {"messages": [user_message]}
result = graph.invoke(initial_state, config = config)

2025-04-24 21:24:29 - agent_router - INFO - Supervisor decided to route to: micro
2025-04-24 21:24:29 - agent_router - INFO - Micro agent is processing...
2025-04-24 21:24:42 - agent_router - INFO - Tool Call: Cypher query generated -> {'answer': '- The `Utils.Utils` module is defined in the `Utils.Utils.Unionfind`, which uses `Utils.Use_Cases.Collection_Deduplication`. This is part of `Utils.Use_Cases` that defines `Utils.Use_Cases.Nearest_Neighbor_Search`, all of which are imported into the `Deduplication.__Main__` module.\n- The `Utils.Utils.Unionfind` uses `Utils.Use_Cases.Collection_Deduplication`; this, in turn, is indirectly used in `Deduplication.__Main__`.\n- The functionality in `Utils.Utils` indirectly influences parts of `Deduplication.__Main__` through its associations with `Utils.Utils.Read_Tsv` and various other modules that are part of `Deduplication`.\n- The `Deduplication.__Main__.Model` defined in `Deduplication.__Main__` uses various implementations in `Deduplicatio

INFO:agent_router:Tool Call: Cypher query generated -> {'answer': '- The `Utils.Utils` module is defined in the `Utils.Utils.Unionfind`, which uses `Utils.Use_Cases.Collection_Deduplication`. This is part of `Utils.Use_Cases` that defines `Utils.Use_Cases.Nearest_Neighbor_Search`, all of which are imported into the `Deduplication.__Main__` module.\n- The `Utils.Utils.Unionfind` uses `Utils.Use_Cases.Collection_Deduplication`; this, in turn, is indirectly used in `Deduplication.__Main__`.\n- The functionality in `Utils.Utils` indirectly influences parts of `Deduplication.__Main__` through its associations with `Utils.Utils.Read_Tsv` and various other modules that are part of `Deduplication`.\n- The `Deduplication.__Main__.Model` defined in `Deduplication.__Main__` uses various implementations in `Deduplication.Lshforest.Lshforest`, `Deduplication.Lshimproved.Lshimproved`, and `Deduplication.Lsh.Lsh`, all of which are connected to `Utils.Utils` through indirect usage pathways.\n- Additio

2025-04-24 21:24:42 - agent_router - INFO - Result type: <class 'langgraph.pregel.io.AddableValuesDict'>


INFO:agent_router:Result type: <class 'langgraph.pregel.io.AddableValuesDict'>


2025-04-24 21:25:14 - agent_router - INFO - Generating text response...


...

INFO:agent_router:Results: {'answer': '- The `Utils.Utils` module is defined in the `Utils.Utils.Unionfind`, which uses `Utils.Use_Cases.Collection_Deduplication`. This is part of `Utils.Use_Cases` that defines `Utils.Use_Cases.Nearest_Neighbor_Search`, all of which are imported into the `Deduplication.__Main__` module.\n- The `Utils.Utils.Unionfind` uses `Utils.Use_Cases.Collection_Deduplication`; this, in turn, is indirectly used in `Deduplication.__Main__`.\n- The functionality in `Utils.Utils` indirectly influences parts of `Deduplication.__Main__` through its associations with `Utils.Utils.Read_Tsv` and various other modules that are part of `Deduplication`.\n- The `Deduplication.__Main__.Model` defined in `Deduplication.__Main__` uses various implementations in `Deduplication.Lshforest.Lshforest`, `Deduplication.Lshimproved.Lshimproved`, and `Deduplication.Lsh.Lsh`, all of which are connected to `Utils.Utils` through indirect usage pathways.\n- Additionally, `Utils.Utils` is conn

2025-04-24 21:25:14 - agent_router - INFO - Question: How is the Utils.Utils module related to the Deduplication.__Main__ module?


INFO:agent_router:Question: How is the Utils.Utils module related to the Deduplication.__Main__ module?


2025-04-24 21:25:25 - agent_router - INFO - Supervisor decided to route to: FINISH


INFO:agent_router:Supervisor decided to route to: FINISH


In [11]:
result['messages']

[HumanMessage(content='How is the Utils.Utils module related to the Deduplication.__Main__ module?', additional_kwargs={}, response_metadata={}, id='371c6701-4857-4a96-812b-f193c75209c3'),
 AIMessage(content="I've decided to route to micro because this is the appropriate next step.", additional_kwargs={}, response_metadata={}, name='supervisor', id='a095fe7d-7f6c-451d-8892-69d8ba7842d3'),
 AIMessage(content="The `Utils.Utils` module plays a crucial role in the `Deduplication.__Main__` module by providing fundamental utilities that aid in document deduplication and nearest neighbor search operations. Here's an overview of their relationship:\n\n1. **Union-Find Operations**: The `UnionFind` class in `Utils.Utils` offers efficient set operations important for managing clusters of duplicates and merging them, which is essential for deduplication.\n\n2. **Locality-Sensitive Hashing (LSH)**: This utility is employed by various modules like `Deduplication.Lshforest.Lshforest`, `Deduplication.

In [12]:
print(result['messages'][-2].content)

The `Utils.Utils` module plays a crucial role in the `Deduplication.__Main__` module by providing fundamental utilities that aid in document deduplication and nearest neighbor search operations. Here's an overview of their relationship:

1. **Union-Find Operations**: The `UnionFind` class in `Utils.Utils` offers efficient set operations important for managing clusters of duplicates and merging them, which is essential for deduplication.

2. **Locality-Sensitive Hashing (LSH)**: This utility is employed by various modules like `Deduplication.Lshforest.Lshforest`, `Deduplication.Lshimproved.Lshimproved`, and `Deduplication.Lsh.Lsh`, supporting deduplication and approximate nearest neighbor searches.

3. **Performance and Logging**: The `Utils.Utils` module contributes to performance monitoring and logging by collaborating with system utilities such as `Psutil` and `Os`, allowing the tracking of resource usage and providing detailed logs.

These connections enhance the main script's capab

# Evaluation

In [57]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())
import importlib

import tools.micro_text_answer as mta

importlib.reload(mta)


db_setup, query_engine = mta.create_rag_system(
    repo_path="../docs/notebooks/assignment-2-mcdonald-s/src",
    embedding_model="text-embedding-3-large",
    llm_model="gpt-4o",
)

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [3]:
docs = db_setup.all_docs

In [4]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=20)

Applying CustomNodeFilter:   0%|          | 0/46 [00:00<?, ?it/s]         Node 90e5fc0a-1b22-4257-b115-53a0683d2530 does not have a summary. Skipping filtering.
Node 248a4442-f40b-4a00-a2b7-36e752fc5274 does not have a summary. Skipping filtering.
Node e316158e-152e-40d9-a195-e01604e84e23 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  13%|█▎        | 6/46 [00:00<00:03, 10.16it/s]Node c84fc18a-4412-4f1e-89ec-9a9f6a31811b does not have a summary. Skipping filtering.
Node 7a5c5031-ae0a-45c5-8b42-c47c2d6cfa27 does not have a summary. Skipping filtering.
Generating Samples: 100%|██████████| 12/12 [00:05<00:00,  2.26it/s]


In [8]:
pd_dataset = dataset.to_pandas()

In [38]:
# pd_dataset.head()

In [ ]:
# test_query = pd_dataset['user_input'][0]
# test_results = query_engine.query(test_query)
# test_results

In [58]:
result_list = []
references_list = []
num = 0
for i in pd_dataset['user_input']:
    num += 1
    print(num)
    results = query_engine.query(i)
    references = [i.page_content for i in query_engine.retrieved_docs]

    result_list.append(results)
    references_list.append(references)

pd_dataset['response'] = result_list
pd_dataset['retrieved_contexts'] = references_list


1
2
3
4
5
6
7
8
9
10
11
12


In [59]:
pd_dataset.head()

,user_input,reference_contexts,reference,synthesizer_name,response,retrieved_contexts
0,What are Bloom Filters used for in data proces...,[The code provided defines several implementat...,Bloom Filters are used to test whether an elem...,single_hop_specifc_query_synthesizer,Bloom Filters are used in data processing prim...,[### Usage:\n\nEach class is instantiated with...
1,How Kirsch-Mitzenmacher Optimization help in B...,[- **Methods**:\n - `add(item)`: \n ...,The Kirsch-Mitzenmacher Optimization in BloomF...,single_hop_specifc_query_synthesizer,**Kirsch-Mitzenmacher Optimization in Bloom Fi...,[2. **BloomFilter Class:**\n - Uses n-grams ...
2,Cud yu pleese explane how the BloomFilter_QF w...,[3. **BloomFilter_Uni_Hash**:\n - Uses **Uni...,The BloomFilter_QF implements a variant using ...,single_hop_specifc_query_synthesizer,The BloomFilter_QF class leverages a quotient ...,[2. **BloomFilter Class:**\n - Uses n-grams ...
3,what dedup.py do?,[The code provided is part of a Python module ...,The code provided is part of a Python module d...,single_hop_specifc_query_synthesizer,"The script `dedup.py`, located at the path `de...","[This script, found at the path `deduplication..."
4,How do tokenization and MinHash signatures con...,[<1-hop>\n\n7. **Method: `tokenize`**:\n - I...,Tokenization contributes to optimizing dedupli...,multi_hop_abstract_query_synthesizer,Tokenization and MinHash signatures play cruci...,[### Overall Functionality:\nThe class provide...


In [45]:
from ragas import EvaluationDataset
import ast

dataset_list = pd_dataset.to_dict(orient="records")
eval_data = EvaluationDataset.from_list(dataset_list)

In [46]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o", model_provider="openai")
# generator_llm = LangchainLLMWrapper(llm)
evaluator_llm = LangchainLLMWrapper(llm)

In [67]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=eval_data,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(mode="recall")],llm=evaluator_llm)
result

Evaluating:  56%|█████▌    | 20/36 [02:51<02:54, 10.89s/it]Exception raised in Job[1]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Evaluating:  58%|█████▊    | 21/36 [03:00<02:31, 10.12s/it]Exception raised in Job[7]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Evaluating: 100%|██████████| 36/36 [04:41<00:00,  7.82s/it]


{'context_recall': 0.8278, 'faithfulness': 0.7533, 'factual_correctness(mode=recall)': 0.6842}

In [69]:
print(pd_dataset.iloc[4]['user_input'])

How do tokenization and MinHash signatures contribute to optimizing deduplication processes in large datasets, and how is performance monitored?


In [70]:
print(pd_dataset.iloc[4]['reference'])

Tokenization contributes to optimizing deduplication processes by splitting a document into words (tokens) and using a `Counter` object to provide word frequencies, which helps in identifying duplicates based on word count resemblance. MinHash signatures are computed using a specified number of hash functions, which are part of the initialization parameters for a class that handles deduplication. These signatures help in efficiently identifying similar documents by mapping bands to document IDs for candidate pairing. Performance is monitored through a memory logging function `log_memory_usage()`, which logs the process's memory usage to debug and optimize resource utilization.


In [71]:
print(pd_dataset.iloc[4]['response'])

Tokenization and MinHash signatures play crucial roles in enhancing the efficiency of deduplication processes in large datasets. Tokenization breaks down documents into smaller components, such as words or tokens, which can then be analyzed for content overlap or similarities. This lays the groundwork for more advanced similarity detection methods, such as MinHash signatures.

MinHash signatures help by creating a compact representation of a document's content through hashing techniques. By generating shingles from tokenized documents and then applying MinHash, documents are translated into a set of signatures. These signatures are far more space-efficient than directly comparing all tokens or words, enabling quick similarity assessments across vast datasets. The use of MinHash allows for the identification of near-duplicates by comparing these signatures rather than entire documents, significantly speeding up the deduplication process.

Performance monitoring and optimization are faci

In [83]:
for i in pd_dataset.iloc[4]['reference_contexts']:
    print(i)

<1-hop>

7. **Method: `tokenize`**:
   - Inputs: A single document string.
   - Functionality: Splits the document into words (tokens) based on spaces and returns a `Counter` object that provides word frequencies.

8. **Method: `word_count_baseline`**:
   - Inputs: A list of document strings and a `threshold` specifying the minimum word overlap fraction required to consider two documents as duplicates.
   - Functionality: Uses word count resemblance to identify duplicates. For every pair of documents, it computes the overlap ratio by comparing the intersection and union of their word frequencies.
   - If the overlap ratio meets or exceeds the given threshold, the document pair is flagged as a duplicate.
   - Outputs: A list of tuples, each containing two documents identified as duplicates.
<2-hop>

2. **Initialization (__init__ method)**:
   - The class is initialized with four parameters:
     - `num_hashes`: Total number of hash functions used for computing MinHash signatures.
     -

In [84]:
for i in pd_dataset.iloc[4]['retrieved_contexts']:
    print(i)

### Overall Functionality:
The class provides a comprehensive approach to deduplication using simple, yet effective methods such as hashing and word/token comparisons. It uses these strategies to both exactly match document contents and also assess similarity using word overlap, making it adaptable for different deduplication scenarios. These baseline methods could serve as a starting point for more sophisticated deduplication algorithms or be directly used in contexts where simple duplicate detection suffices.
4. **Removing Exact Duplicates (`remove_duplicates`)**:
    - This method processes a dictionary of documents, identifies exact duplicates, and stores only unique documents.
    - It updates `unique_docs` with non-duplicate documents and records duplicate relationships in `exact_duplicates`.

5. **Computing MinHash Signatures (`compute_minhash_signatures`)**:
    - The method begins by removing exact duplicates.
    - It cleans the documents using a utility function, then create

In [73]:
result.traces

[{'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.83)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(1.0)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.89)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(1.0)},
 {'context_recall': 0.5, 'faithfulness': {}, 'factual_correctness': np.float64(0.36)},
 {'context_recall': 0.8333333333333334, 'faithfulness': {}, 'factual_correctness': np.float64(0.58)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.82)},
 {'context_recall': 0.6, 'faithfulness': {}, 'factual_correctness': np.float64(0.58)},
 {'context_recall': 0.5, 'faithfulness': 0.8846153846153846, 'factual_correctness': np.float64(0.67)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.64)},
 {'context_recall': 1.0, 'faithfulness': 0.76, 'factual_correctness': np.float64(0.44)},
 {'context_r

In [ ]:
result

[{'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.83)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(1.0)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.89)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(1.0)},
 {'context_recall': 0.5, 'faithfulness': {}, 'factual_correctness': np.float64(0.36)},
 {'context_recall': 0.8333333333333334, 'faithfulness': {}, 'factual_correctness': np.float64(0.58)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.82)},
 {'context_recall': 0.6, 'faithfulness': {}, 'factual_correctness': np.float64(0.58)},
 {'context_recall': 0.5, 'faithfulness': 0.8846153846153846, 'factual_correctness': np.float64(0.67)},
 {'context_recall': 1.0, 'faithfulness': {}, 'factual_correctness': np.float64(0.64)},
 {'context_recall': 1.0, 'faithfulness': 0.76, 'factual_correctness': np.float64(0.44)},
 {'context_r

In [23]:
query_engine.retrieved_docs

[Document(id='c9927211-37b5-4885-af49-44556c55348a', metadata={'source': 'deduplication.bloom_filter.py'}, page_content='Each class is designed to balance space efficiency with computation speed while providing varying optimizations to bloom filter operations. These variations encompass foundational techniques for reducing hash computations required for membership checks, improving speed, and potentially lowering false positive rates under different scenarios.'),
 Document(id='d4335b4a-ad72-4206-939d-54ed337376ef', metadata={'source': 'deduplication.bloom_filter.py'}, page_content='The code provided defines several implementations of the Bloom Filter, a probabilistic data structure used to test whether an element is a member of a set. Bloom Filters can efficiently check for membership with a certain probability of false positives but no false negatives, making them useful in applications where space efficiency and speed are crucial.\n\n### Overview of Classes\n\n1. **BloomFilter**: \n 

In [8]:
print(query_engine.query('What is the Utils.Utils module?'))

The `Utils.Utils` module in the provided context is a vital component of a Python package designed for handling tasks related to document processing and deduplication. This module, located in `utils/utils.py`, encompasses a range of utility functions and a class, all tailored towards text processing, document comparison, and executing simple set operations effectively. 

Key features within the module include:

1. **Text Cleaning Functionality**:
   - The function `clean_document(text)` is designed to sanitize and standardize text by converting it to lowercase and stripping non-alphabetical characters, ensuring only lowercase letters and spaces are retained. This pre-processing is crucial for consistent text analysis, especially for tasks like deduplication and text clustering.

2. **Union-Find Data Structure**:
   - The module introduces a `UnionFind` class which implements a classic data structure facilitating efficient union and find operations. The `UnionFind` structure is remarkab

In [9]:
query_engine.retrieved_docs

[Document(id='6f2b6c5b-c95a-4f73-bd2c-f1baf70a31ce', metadata={'source': 'utils.utils.py'}, page_content='The provided code in `utils/utils.py` consists of several utility functions and a class, primarily geared toward text processing, document comparison, and simple set operations. Here\'s a detailed description of each component:\n\n1. **Imports**: The script imports several modules such as `hashlib`, `re`, `xxhash`, and `Counter` from `collections`. These are used for hashing, regular expressions, and counting elements, respectively.\n\n2. **Function: `clean_document(text)`**:\n   - Purpose: Cleans and normalizes a given text document by converting it to lowercase and removing non-alphabetical characters, preserving only lowercase letters and spaces.\n   - Process: It uses a regular expression to filter out any characters that aren\'t lowercase letters or spaces.\n   - Example Usage: Transforming "Hello, World!123" into "hello world".'),
 Document(id='db96b8e8-4304-452a-a66f-0451aea

# Macro Output Test

In [2]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\JaeHoBahng\\Desktop\\Georgetown\\2025_Spring\\DSAN_6725\\project\\spring-2025-final-project-project-group-2\\src')
print(os.getcwd())

c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\docs\notebooks
c:\Users\JaeHoBahng\Desktop\Georgetown\2025_Spring\DSAN_6725\project\spring-2025-final-project-project-group-2\src


In [ ]:
from tools.macro import tools as mat
importlib.reload(mat)

from langgraph.prebuilt import create_react_agent

mat.repo_path = "../cloned_repository/src"

tools = [
    mat.generate_text_response,
    mat.generate_mermaid_diagram
]

system_prompt = f"""
You are a macro-level strategic agent. You excel at:
- Understanding the big picture
- Creating high-level plans and strategies
- Breaking down complex problems into manageable parts
- Identifying key objectives and priorities

You are given the following tools.
{tools}

You will use each tool only once and the guidelines of the tools are as follows:
- You will always use the generate_text_response tool to get an initial answer.
- You will always use the generate_mermaid_diagram tool to get a final answer.


Focus on planning and strategic thinking. For detailed implementation, 
defer to the micro agent.

Always provide clear, structured responses with your analysis and recommendations. Also respond complete once the generation is completed.
"""

from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o", model_provider="openai")

agent = create_react_agent(llm, tools=tools, prompt=system_prompt)

In [10]:
answer = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Give me an overall summary of the repository with a visualization as well",
            }
        ],
        "goto": "supervisor",
    }
)

2025-04-26 14:17:20 - agent_router - INFO - Generating text response for codebase at: ../cloned_repository/src
2025-04-26 14:17:33 - agent_router - INFO - Text response generated successfully using LLM
2025-04-26 14:17:39 - agent_router - INFO - Generating mermaid diagram from text response
2025-04-26 14:17:42 - agent_router - INFO - Mermaid diagram saved to codebase_structure.md
2025-04-26 14:17:42 - agent_router - INFO - Mermaid diagram HTML saved to codebase_structure.html


In [15]:
print(answer['messages'][-1].content)

### Codebase Summary and Structure

The provided codebase is organized into main modules that focus on deduplication and utility functions to support these operations. The primary directory is dedicated to various deduplication strategies, implementing both basic and advanced techniques, while a set of utility modules aids in preprocessing and visualization tasks.

#### Main Modules & Purposes:
1. **deduplication/**
   - **bloom_filter.py**: Bloom Filter implementation for efficient set membership checks.
   - **dedup.py**: Baseline deduplication using hash comparisons and token overlap.
   - **LSH.py**: Locality Sensitive Hashing with MinHash for duplicate detection.
   - **LSHForest.py**: Uses multiple trees for robustness in LSH-based detection.
   - **LSHImproved.py**: Multi-probe techniques in LSH to improve efficiency.

2. **utils/**
   - **use_cases.py**: Defines practical applications of deduplication strategies.
   - **utils.py**: Helper functions including document cleaning a

In [20]:
import json
# Standard Library
import os
from typing import Annotated, Any, Dict, List

# Third-Party Libraries
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.language_models import BaseChatModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
# Internal Utilities
from utils import get_logger
from utils.filesystem import list_python_files, read_code_file
from utils.repo import generate_repo_tree

# Initialization
# Load environment variables from .env file
load_dotenv()

# Set up module-wide logger
logger = get_logger()

repo_path = "../cloned_repository/src"


# @tool
# def generate_text_response(question: str):
#     """
#     Reads in the codebase and generate a short to medium length summary in natural language that captures the main logical modules of the codebase. Output text that is preferably good for translating to mermaid diagrams, and good for reading.

#     Args:
#         path (str): The path to the codebase

#     Returns:
#         str: Natural language answer to the predefined question.
#     """
#     logger.info(f"Generating text response for codebase at: {repo_path}")

    # Initialize the language model
llm = init_chat_model("gpt-4.1-mini", model_provider="openai")

# Get all Python files in the repository
python_files = list_python_files(repo_path)

# if not python_files:
#     return "No Python files found in the specified path."

# Generate repository tree structure
repo_tree = generate_repo_tree(repo_path)

# Read the content of each Python file (limit to a reasonable number to avoid token limits)
file_contents = {}
max_files = 20  # Limit to avoid token limits
for file_path in python_files:
    full_path = os.path.join(repo_path, file_path)
    content = read_code_file(full_path)
    file_contents[file_path] = content

# Create a summary of file contents for the LLM
file_summaries = []
for file_path, content in file_contents.items():
    # Truncate very large files to avoid token limits
    # if len(content) > 2000:
    #     content = content[:1000] + "\n...[content truncated]...\n" + content[-1000:]

    file_summaries.append(f"File: {file_path}\n```python\n{content}\n```\n")

# Join file summaries with a limit to avoid token limits
file_content_text = "\n\n".join(file_summaries)

# Create the prompt for the LLM
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""You qre an expert in generating well composed, extremely detailed reports based on the codebase and its structure.
             Provide an extremely detailed review of the codebase.
             Make it in paragraph form including its architecture and key components, detailed agentic workflow, flaws, points of improvement, etc.
             Only use bullet points when necessary.
             Generate the report based on the user input.""",
        ),
        (
            "human",
            """
    
    Repository structure:
    {repo_tree}
    
    Here are the contents of key files:
    {file_contents}
    
    question: 
    {question}
    """,
        ),
    ]
)

# Generate the analysis using the LLM
chain = prompt | llm | StrOutputParser()


In [21]:
analysis = chain.invoke(
    {"repo_tree": repo_tree, "file_contents": file_content_text, "question": "Give me an extremely detailed academic report based on the codebase given. Give it to me in mainly paragraph format and bullet points only when necessary."}
)

In [22]:
print(analysis)

The analyzed codebase presents a sophisticated application designed to perform interactive codebase analysis through a web interface, leveraging advanced language models, graph representations, and retrieval-augmented generation (RAG) methods. The architecture is modular and carefully divided into core components that collectively offer cloning, visualizing, querying, and interpreting GitHub repositories in relation to their internal structure and semantic relationships between software entities. At its core, the system embodies a LangGraph-based agent router architecture that collaboratively orchestrates macro and micro level agents to handle high-level overviews and detailed code analysis respectively.

### Architecture and Key Components

The repository features a clear, hierarchical file structure centered around several key directories: `src/`, which contains the main application code; `agent_router/` housing the graph router logic and node definitions; `app/` which includes sessi